# 7-3 Join many-many tables

## Add reservations data by hotels to customer master

### Pandas

In [1]:
import pandas as pd

In [2]:
pd_reservation = pd.read_parquet("../../data/reservation.parquet")
pd_hotel       = pd.read_parquet("../../data/hotel.parquet")
pd_customer    = pd.read_parquet("../../data/customer.parquet")

In [3]:
pd_reservation.head()

,reservation_id,hotel_id,customer_id,reserved_at,checkin_date,checkout_date,length_of_stay,people_num,total_price,status,canceled_at
0,1,2460,53431,2013-12-31 07:00:14+09:00,2014-12-31 00:00:00+09:00,2015-01-03 00:00:00+09:00,3,2,37800,reserved,NaT
1,2,962,488390,2013-12-31 08:23:35+09:00,2014-12-31 00:00:00+09:00,2015-01-02 00:00:00+09:00,2,3,42000,reserved,NaT
2,3,558,341335,2013-12-31 09:02:05+09:00,2014-12-31 00:00:00+09:00,2015-01-01 00:00:00+09:00,1,2,20400,reserved,NaT
3,4,3666,398981,2013-12-31 23:44:54+09:00,2014-12-31 00:00:00+09:00,2015-01-01 00:00:00+09:00,1,4,39600,reserved,NaT
4,5,2180,220381,2014-01-01 02:47:50+09:00,2014-12-31 00:00:00+09:00,2015-01-01 00:00:00+09:00,1,3,16500,reserved,NaT


In [4]:
pd_hotel.head()

,hotel_id,hotel_name,hotel_type,address_prefecture,address_city,address_town,address_zipcode,unit_price,user_rating,tag_001,...,tag_021,tag_022,tag_023,tag_024,tag_025,tag_026,tag_027,tag_028,tag_029,tag_030
0,1,北飯岡ホテル,ビジネスホテル,岩手県,盛岡市,北飯岡,020-0857,10300,3.256591,0,...,0,1,<NA>,0,0,1,0,0,0,0
1,2,西二条南温泉ホテル,リゾートホテル,北海道,中川郡美深町,西二条南,098-2242,6800,3.453642,0,...,0,0,0,<NA>,1,0,0,0,0,0
2,3,小屋敷ペンション,民宿,青森県,黒石市,小屋敷,036-0511,18600,1.381796,<NA>,...,0,0,0,1,<NA>,<NA>,0,0,0,0
3,4,中後町民宿,民宿,愛知県,碧南市,中後町,447-0042,4900,2.090353,0,...,0,0,0,0,0,0,0,0,1,0
4,5,鵜沼台ホテル,ビジネスホテル,岐阜県,各務原市,鵜沼台,509-0121,9000,2.771490,0,...,0,0,0,0,0,1,1,1,0,0


In [5]:
pd_customer.head()

,customer_id,name,age,sex,address_prefecture,address_city,address_town,address_zipcode
0,1,山田 裕太,75,<NA>,岐阜県,岐阜市,鷺山清洲町,502-0853
1,2,藤井 稔,83,M,大阪府,豊能郡能勢町,地黄,563-0121
2,3,青木 太一,62,M,佐賀県,佐賀市,本庄町袋,840-0023
3,4,渡辺 裕太,28,M,福島県,喜多方市,豊川町高堂太,966-0911
4,5,渡辺 明美,62,F,兵庫県,西宮市,津門西口町,663-8231


In [6]:
import numpy as np

(
    pd_customer
    .merge(
        pd_reservation
        .merge(
            pd_hotel[["hotel_id", "hotel_type"]],
            on="hotel_id",
            how="left",
        )
        .assign(
            ryokan_cnt=lambda df: np.where(df.hotel_type == "旅館", 1, 0),
            resort_hotel_cnt=lambda df: np.where(df.hotel_type == "リゾートホテル", 1, 0),
            business_hotel_cnt=lambda df: np.where(df.hotel_type == "ビジネスホテル", 1, 0),
            minsyuku_cnt=lambda df: np.where(df.hotel_type == "民宿", 1, 0),
        )
        .groupby("customer_id")[["ryokan_cnt",        "resort_hotel_cnt",
                                "business_hotel_cnt", "minsyuku_cnt"]].sum(),
        on="customer_id",
        how="left"
    )
    .fillna({
        "ryokan_cnt": 0,         "resort_hotel_cnt": 0,
        "business_hotel_cnt": 0, "minsyuku_cnt": 0
    })
)

,customer_id,name,age,sex,address_prefecture,address_city,address_town,address_zipcode,ryokan_cnt,resort_hotel_cnt,business_hotel_cnt,minsyuku_cnt
0,1,山田 裕太,75,<NA>,岐阜県,岐阜市,鷺山清洲町,502-0853,1.0,0.0,1.0,4.0
1,2,藤井 稔,83,M,大阪府,豊能郡能勢町,地黄,563-0121,1.0,1.0,1.0,3.0
2,3,青木 太一,62,M,佐賀県,佐賀市,本庄町袋,840-0023,2.0,2.0,0.0,3.0
3,4,渡辺 裕太,28,M,福島県,喜多方市,豊川町高堂太,966-0911,1.0,0.0,2.0,1.0
4,5,渡辺 明美,62,F,兵庫県,西宮市,津門西口町,663-8231,2.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
499995,499996,高橋 洋介,26,M,山形県,山形市,下東山,990-2232,1.0,0.0,1.0,2.0
499996,499997,山口 治,42,M,岩手県,一関市,室根町津谷川,029-1211,0.0,0.0,1.0,2.0
499997,499998,石井 拓真,39,M,山形県,山形市,桜ケ丘,990-2376,0.0,0.0,0.0,1.0
499998,499999,坂本 くみ子,66,F,群馬県,富岡市,曽木,370-2315,3.0,0.0,1.0,0.0


### Polars

In [7]:
import polars as pl

In [8]:
pl_reservation = pl.read_parquet("../../data/reservation.parquet")
pl_hotel       = pl.read_parquet('../../data/hotel.parquet')
pl_customer    = pl.read_parquet("../../data/customer.parquet")

In [9]:
pl_reservation.head()

reservation_id,hotel_id,customer_id,reserved_at,checkin_date,checkout_date,length_of_stay,people_num,total_price,status,canceled_at
i64,i64,i64,"datetime[ns, Asia/Tokyo]","datetime[ns, Asia/Tokyo]","datetime[ns, Asia/Tokyo]",i64,i64,i64,str,"datetime[ns, Asia/Tokyo]"
1,2460,53431,2013-12-31 07:00:14 JST,2014-12-31 00:00:00 JST,2015-01-03 00:00:00 JST,3,2,37800,"""reserved""",null
2,962,488390,2013-12-31 08:23:35 JST,2014-12-31 00:00:00 JST,2015-01-02 00:00:00 JST,2,3,42000,"""reserved""",null
3,558,341335,2013-12-31 09:02:05 JST,2014-12-31 00:00:00 JST,2015-01-01 00:00:00 JST,1,2,20400,"""reserved""",null
4,3666,398981,2013-12-31 23:44:54 JST,2014-12-31 00:00:00 JST,2015-01-01 00:00:00 JST,1,4,39600,"""reserved""",null
5,2180,220381,2014-01-01 02:47:50 JST,2014-12-31 00:00:00 JST,2015-01-01 00:00:00 JST,1,3,16500,"""reserved""",null


In [10]:
pl_hotel.head()

hotel_id,hotel_name,hotel_type,address_prefecture,address_city,address_town,address_zipcode,unit_price,user_rating,tag_001,tag_002,tag_003,tag_004,tag_005,tag_006,tag_007,tag_008,tag_009,tag_010,tag_011,tag_012,tag_013,tag_014,tag_015,tag_016,tag_017,tag_018,tag_019,tag_020,tag_021,tag_022,tag_023,tag_024,tag_025,tag_026,tag_027,tag_028,tag_029,tag_030
i64,str,str,str,str,str,str,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1,"""北飯岡ホテル""","""ビジネスホテル""","""岩手県""","""盛岡市""","""北飯岡""","""020-0857""",10300,3.256591,0,0,0,0,0,0,0,0,0,0,0,0,0,1,null,0,0,1,1,0,0,1,null,0,0,1,0,0,0,0
2,"""西二条南温泉ホテル""","""リゾートホテル""","""北海道""","""中川郡美深町""","""西二条南""","""098-2242""",6800,3.453642,0,0,0,null,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,null,1,0,0,0,0,0
3,"""小屋敷ペンション""","""民宿""","""青森県""","""黒石市""","""小屋敷""","""036-0511""",18600,1.381796,null,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,0,0,0,1,null,null,0,0,0,0
4,"""中後町民宿""","""民宿""","""愛知県""","""碧南市""","""中後町""","""447-0042""",4900,2.090353,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5,"""鵜沼台ホテル""","""ビジネスホテル""","""岐阜県""","""各務原市""","""鵜沼台""","""509-0121""",9000,2.77149,0,0,0,1,0,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0


In [11]:
pl_customer.head()

customer_id,name,age,sex,address_prefecture,address_city,address_town,address_zipcode
i64,str,i64,str,str,str,str,str
1,"""山田 裕太""",75,null,"""岐阜県""","""岐阜市""","""鷺山清洲町""","""502-0853"""
2,"""藤井 稔""",83,"""M""","""大阪府""","""豊能郡能勢町""","""地黄""","""563-0121"""
3,"""青木 太一""",62,"""M""","""佐賀県""","""佐賀市""","""本庄町袋""","""840-0023"""
4,"""渡辺 裕太""",28,"""M""","""福島県""","""喜多方市""","""豊川町高堂太""","""966-0911"""
5,"""渡辺 明美""",62,"""F""","""兵庫県""","""西宮市""","""津門西口町""","""663-8231"""


In [12]:
(
    pl_customer
    .join(
        pl_reservation
        .join(
            pl_hotel.select(["hotel_id", "hotel_type"]),
            on="hotel_id",
            how="left",
        )
        .group_by("customer_id")
        .agg([
            pl.col("reservation_id").filter(pl.col("hotel_type") == "旅館").len().alias("ryokan_cnt"),
            pl.col("reservation_id").filter(pl.col("hotel_type") == "リゾートホテル").len().alias("resort_hotel_cnt"),
            pl.col("reservation_id").filter(pl.col("hotel_type") == "ビジネスホテル").len().alias("business_hotel_cnt"),
            pl.col("reservation_id").filter(pl.col("hotel_type") == "民宿").len().alias("minsyuku_cnt"),
        ]),
        on="customer_id",
        how="left",
    )
    .with_columns(
        ryokan_cnt=pl.col("ryokan_cnt").fill_null(0),
        resort_cnt=pl.col("resort_hotel_cnt").fill_null(0),
        business_hotel_cnt=pl.col("business_hotel_cnt").fill_null(0),
        minsyuku_cnt=pl.col("minsyuku_cnt").fill_null(0)
    )
)

customer_id,name,age,sex,address_prefecture,address_city,address_town,address_zipcode,ryokan_cnt,resort_hotel_cnt,business_hotel_cnt,minsyuku_cnt,resort_cnt
i64,str,i64,str,str,str,str,str,u32,u32,u32,u32,u32
1,"""山田 裕太""",75,null,"""岐阜県""","""岐阜市""","""鷺山清洲町""","""502-0853""",1,0,1,4,0
2,"""藤井 稔""",83,"""M""","""大阪府""","""豊能郡能勢町""","""地黄""","""563-0121""",1,1,1,3,1
3,"""青木 太一""",62,"""M""","""佐賀県""","""佐賀市""","""本庄町袋""","""840-0023""",2,2,0,3,2
4,"""渡辺 裕太""",28,"""M""","""福島県""","""喜多方市""","""豊川町高堂太""","""966-0911""",1,0,2,1,0
5,"""渡辺 明美""",62,"""F""","""兵庫県""","""西宮市""","""津門西口町""","""663-8231""",2,1,0,1,1
…,…,…,…,…,…,…,…,…,…,…,…,…
499996,"""高橋 洋介""",26,"""M""","""山形県""","""山形市""","""下東山""","""990-2232""",1,0,1,2,0
499997,"""山口 治""",42,"""M""","""岩手県""","""一関市""","""室根町津谷川""","""029-1211""",0,0,1,2,0
499998,"""石井 拓真""",39,"""M""","""山形県""","""山形市""","""桜ケ丘""","""990-2376""",0,0,0,1,0
